## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [35]:
from keras.datasets import cifar10
from resnet_builder import resnet # 這是從 resnet_builder.py 中直接 import 撰寫好的 resnet 函數
from keras.models import Model
from keras.optimizers import Adam,RMSprop
from keras.utils import to_categorical
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau

In [32]:
(x_train,y_train),(x_test,y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0],'train samples')
print(x_test.shape[0],'test samples')

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train = x_train / 255
x_test = x_test / 255

y_train = to_categorical(y_train,10)
y_test = to_categorical(y_test,10)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [33]:
model = resnet(input_shape=(32,32,3))
model.summary()
model.compile(loss='categorical_crossentropy',optimizer=Adam(), metrics=['accuracy'])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_218 (Conv2D)             (None, 32, 32, 16)   448         input_8[0][0]                    
__________________________________________________________________________________________________
batch_normalization_197 (BatchN (None, 32, 32, 16)   64          conv2d_218[0][0]                 
__________________________________________________________________________________________________
activation_197 (Activation)     (None, 32, 32, 16)   0           batch_normalization_197[0][0]    
__________________________________________________________________________________________________
conv2d_219

In [39]:
def lr_sch(epoch):
#200 total
    if epoch <50:
        return 1e-3
    if 50<=epoch<100:
        return 1e-4
    if epoch>=100:
        return 1e-5
lr_scheduler = LearningRateScheduler(lr_sch)
lr_reducer = ReduceLROnPlateau(monitor='val_acc',factor=0.2,patience=5,
mode='max',min_lr=1e-3)
callbacks = [lr_scheduler,lr_reducer]

In [42]:
batch_size = 32 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 10 # 訓練整個資料集共 30個循環

#train_datagen  = ImageDataGenerator(rotation_range=30,width_shift_range=0.2,height_shift_range=0.2,
#                              shear_range=0.2,zoom_range=0.2,channel_shift_range=10,
#                                   horizontal_flip=True,)

#train_datagen.fit(x_train)
model.fit(x_train,y_train,batch_size=batch_size
          ,epochs=epochs,verbose=1,validation_data=(x_test,y_test),callbacks=callbacks)

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 1103s 22ms/step - loss: 1.7753 - acc: 0.5160 - val_loss: 1.6495 - val_acc: 0.5385
Epoch 2/10
50000/50000 [==============================] - 1051s 21ms/step - loss: 1.2748 - acc: 0.6608 - val_loss: 1.7807 - val_acc: 0.5309
Epoch 3/10
50000/50000 [==============================] - 1110s 22ms/step - loss: 1.0864 - acc: 0.7169 - val_loss: 1.1240 - val_acc: 0.7054
Epoch 4/10
50000/50000 [==============================] - 1082s 22ms/step - loss: 0.9707 - acc: 0.7602 - val_loss: 1.2273 - val_acc: 0.6792
Epoch 5/10
50000/50000 [==============================] - 1088s 22ms/step - loss: 0.8865 - acc: 0.7849 - val_loss: 1.6547 - val_acc: 0.6068
Epoch 6/10
50000/50000 [==============================] - 1091s 22ms/step - loss: 0.8241 - acc: 0.8103 - val_loss: 1.2803 - val_acc: 0.6701
Epoch 7/10
50000/50000 [==============================] - 1067s 21ms/step - loss: 0.7780 - acc: 0.8219 - val_l

In [ ]:
score = model.evaluate(x_test,y_test,verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])